<a href="https://colab.research.google.com/github/avidnerd/DriverFatigueDetector/blob/main/driver_fatigue_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

X = np.load('/content/drive/MyDrive/subhi/saved_data/X1.npy')
y = np.load('/content/drive/MyDrive/subhi/saved_data/y1.npy')


In [ ]:
!pip install EMD-signal

In [ ]:
!pip install tensorflow==0.9.1
!pip install min2net==1.0.0

In [ ]:
from PyEMD import EMD
import numpy as np

emd = EMD()
data = []

#creating the IMFs
time = np.array([[i for i in range(2501)] for i in range(6)])
for i in range(len(drowsy)):
    print(drowsy[i].shape)
    print(T.shape)
    imf = emd(drowsy[i], time)
    data.append(imf)

data = np.array(data)

num_samples = len(data[0][0])
num_imfs = 0
num_frames = data.shape[0]
num_channels = data.shape[1]

#finding the maximum number of IMFs
for i in range(num_frames):
    for j in range(num_channels):
        item_imf = np.array(data[i, j])
        if item_imf.shape[1] > num_imfs:
            num_imfs = item_imf.shape[1]

data_trans = np.zeros([num_samples, num_imfs, num_frames, num_channels])
for i in range(num_frames):
    for j in range(num_channels):
        item_imf = np.array(data[i, j])
        data_trans[:, :item_imf.shape[1], i, j] = item_imf

data = data_trans

set_real_FRMs_idx = [0, 1, 2, 3, 4]
num_arti_FRMs = 500
set_real_IMFs = data
set_real_IMFs_idx = [0, 1, 2]

def create_artifical_frame(set_real_FRMs_idx: list, num_arti_FRMs: int, set_real_IMFs: np.array, set_real_IMFs_idx: list):
    num_samples, num_channels = set_real_IMFs.shape[0], set_real_IMFs.shape[-1]

    arti_FRMs = np.zeros([num_arti_FRMs, num_samples, num_channels])
    for idx_FRM in range(num_arti_FRMs):
        max_arti_IMFs = len(set_real_IMFs_idx)
        item_FRM_IMFs = np.zeros([num_samples, max_arti_IMFs, num_channels])

        seq_FRMs_idx = []
        while(len(seq_FRMs_idx) < max_arti_IMFs):
            seq_FRMs_idx.extend(np.random.permutation(len(set_real_FRMs_idx)))

        seq_FRMs_idx = [int(item) for item in seq_FRMs_idx]

        seq_FRMs_idx = seq_FRMs_idx[:max_arti_IMFs]

        for idx_IMF in range(max_arti_IMFs):
            for idx_channel in range(num_channels):
                idx_IMF_real = set_real_IMFs_idx[idx_IMF]
                idx_FRM_real = set_real_FRMs_idx[seq_FRMs_idx[idx_IMF]]
                selected_IMF = set_real_IMFs[:,
                                             idx_IMF_real, idx_FRM_real, idx_channel]
                item_FRM_IMFs[:, idx_IMF, idx_channel] = selected_IMF

        arti_FRMs[idx_FRM, :, :] = np.sum(item_FRM_IMFs, 1)
    return arti_FRMs

arti_frames = create_artifical_frame(set_real_FRMs_idx,
                                    num_arti_FRMs,
                                    set_real_IMFs,
                                    set_real_IMFs_idx)

drowsy = np.concatenate((drowsy, arti_frames), 0)

In [ ]:
X = np.concatenate((alert,drowsy,transitioning),axis=0)
y = np.array([0]*len(alert)+[1]*len(drowsy)+[2]*len(transitioning))

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
train_ratio = 0.80
validation_ratio = 0.10
test_ratio = 0.10

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))

y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [ ]:
import gc
del X
del y
gc.collect()

In [ ]:
print(X_val.shape)

In [ ]:
print(X_train.shape)
X_train = X_train.reshape((19870, 2501, 5))
X_test = X_test.reshape((2484, 2501, 5))
X_val = X_val.reshape((2484, 2501, 5))
print(X_train.shape)

In [ ]:
!pip install optuna

In [ ]:
!pip install mne

In [ ]:
print(y_train)

In [ ]:
import io
from PIL import Image
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import numpy as np
import matplotlib.pyplot as plt
finX_train = []
# Load the EEG data
for i in range(len(X_train)):
    data = X_train[i]
    print(y_train[i])
    # Extract the EEG data from the data
    channel_index = 2
    data = data[:, channel_index]

    spg, freq, t, im = plt.specgram(data, NFFT=256, cmap='jet',Fs=256, noverlap=128)

    normalized_spg = (spg - np.min(spg)) / (np.max(spg) - np.min(spg))

    cmap = plt.get_cmap('jet')
    rgb_image = cmap(normalized_spg)

    rgb_image = rgb_image[:,:,:3]
    plt.xlabel('Time (samples)')
    plt.ylabel('Frequency (Hz)')
    plt.title('Time-Frequency Spectrogram of EEG Data')
    rgb_images_array = np.array(rgb_image)
    print(rgb_images_array.shape)
    print(rgb_images_array)
    finX_train.append(rgb_images_array)


In [ ]:
finX_train = np.array(finX_train)
# finX_test = np.array(finX_test)
# finX_val = np.array(finX_val)

In [ ]:
import numpy as np
np.save('/content/drive/MyDrive/subhi/saved_data/finX_train_alpha.npy', finX_train)
np.save('/content/drive/MyDrive/subhi/saved_data/finy_train_alpha.npy', y_train)

In [ ]:
import io
from PIL import Image
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import numpy as np
import matplotlib.pyplot as plt
finX_test = []
# Load the EEG data
for i in range(len(X_test)):
    data = X_test[i]
    # Extract the EEG data from the data
    channel_index = 2
    data = data[:, channel_index]

    spg, freq, t, im = plt.specgram(data, NFFT=256, cmap='jet',Fs=256, noverlap=128)

    normalized_spg = (spg - np.min(spg)) / (np.max(spg) - np.min(spg))

    cmap = plt.get_cmap('jet')
    rgb_image = cmap(normalized_spg)

    rgb_image = rgb_image[:,:,:3]

    rgb_images_array = np.array(rgb_image)
    print(rgb_images_array.shape)
    print(rgb_images_array)
    finX_test.append(rgb_images_array)

In [ ]:
import io
from PIL import Image
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import numpy as np
import matplotlib.pyplot as plt
finX_val = []
# Load the EEG data
for i in range(len(X_val)):
    data = X_val[i]
    # Extract the EEG data from the data
    channel_index = 2
    data = data[:, channel_index]

    spg, freq, t, im = plt.specgram(data, NFFT=256, cmap='jet',Fs=256, noverlap=128)

    normalized_spg = (spg - np.min(spg)) / (np.max(spg) - np.min(spg))

    cmap = plt.get_cmap('jet')
    rgb_image = cmap(normalized_spg)

    rgb_image = rgb_image[:,:,:3]

    rgb_images_array = np.array(rgb_image)
    print(rgb_images_array.shape)
    print(rgb_images_array)
    finX_val.append(rgb_images_array)

In [ ]:
finX_train = np.array(finX_train)
finX_test = np.array(finX_test)
finX_val = np.array(finX_val)

In [ ]:
np.save('/content/drive/MyDrive/subhi/saved_data/finX_train_alpha.npy', finX_train)
np.save('/content/drive/MyDrive/subhi/saved_data/finX_test_alpha.npy', finX_test)
np.save('/content/drive/MyDrive/subhi/saved_data/finX_val_alpha.npy', finX_val)
np.save('/content/drive/MyDrive/subhi/saved_data/y_train_alpha.npy', y_train)
np.save('/content/drive/MyDrive/subhi/saved_data/y_test_alpha.npy', y_test)
np.save('/content/drive/MyDrive/subhi/saved_data/y_val_alpha.npy', y_val)

In [ ]:
import numpy as np
finX_train = np.load('/content/drive/MyDrive/subhi/saved_data/finX_train.npy')
finX_test = np.load('/content/drive/MyDrive/subhi/saved_data/finX_test.npy')
finX_val = np.load('/content/drive/MyDrive/subhi/saved_data/finX_val.npy')

In [ ]:
import optuna
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Reshape, Flatten, Dense, BatchNormalization, LSTM
from keras import metrics

def create_model(trial):
    # Define the CNN part of the model
    model = Sequential()
    model.add(Conv2D(filters=trial.suggest_int('filters_1', 16, 64), kernel_size=trial.suggest_int('kernel_size_1', 2, 5), activation='relu', input_shape=(129, 18, 3)))
    model.add(Conv2D(filters=trial.suggest_int('filters_2', 16, 64), kernel_size=trial.suggest_int('kernel_size_2', 2, 5), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(trial.suggest_float('dropout_1', 0.1, 0.5)))
    model.add(MaxPooling2D(pool_size=trial.suggest_int('pool_size_1', 2, 4)))

    model.add(Conv2D(filters=trial.suggest_int('filters_5', 16, 64), kernel_size=trial.suggest_int('kernel_size_5', 1, 2), activation='relu'))

    model.add(Flatten())

    model.add(Reshape((1, -1)))
    model.add(LSTM(trial.suggest_int('lstm_units', 30, 128), return_sequences=False))
    model.add(Flatten())
    model.add(Dense(2, activation='softmax'))

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy', metrics.AUC(name='my_auc')])
    return model

def objective(trial):
    model = create_model(trial)

    history = model.fit(finX_train, y_train, validation_data=(finX_val, y_val), epochs=15, batch_size=32, verbose=1)

    return history.history['val_accuracy'][-1]

study = optuna.create_study(direction='maximize')

study.optimize(objective, n_trials=50)

print("Best trial:")
trial_ = study.best_trial

print(" Value: {}".format(trial_.value))

print(" Params: ")
for key, value in trial_.params.items():
    print("    {}: {}".format(key, value))


In [ ]:
y_train = np.load('/content/drive/MyDrive/subhi/saved_data/y_train.npy')
y_test = np.load('/content/drive/MyDrive/subhi/saved_data/y_test.npy')
y_val = np.load('/content/drive/MyDrive/subhi/saved_data/y_val.npy')

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Reshape, ReLU, Dropout, Flatten, Dense, BatchNormalization, TimeDistributed, LSTM
from keras import metrics

model = Sequential()
model.add(Conv2D(filters=64, kernel_size=4, input_shape=(129, 18, 3)))
model.add(BatchNormalization())
model.add(ReLU())

model.add(Conv2D(filters=51, kernel_size=4))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=4))

model.add(Conv2D(filters=20, kernel_size=2, activation='relu'))
model.add(BatchNormalization())
model.add(ReLU())

model.add(Flatten())

model.add(Reshape((1, -1)))
model.add(LSTM(58, return_sequences=False))
model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(2, activation='sigmoid'))

model.build(input_shape=(129, 18, 3))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[ metrics.MeanSquaredError(name='my_mse'),
                                                                           metrics.AUC(name='my_auc'),])
model.summary()


In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Reshape, ReLU, Dropout, Flatten, Dense, BatchNormalization, TimeDistributed, LSTM
from keras import metrics

model = Sequential()
model.add(Conv2D(filters=64, kernel_size=4, input_shape=(129, 18, 3)))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Conv2D(filters=32, kernel_size=4))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=4))
model.add(Conv2D(filters=16, kernel_size=2, activation='relu'))
model.add(BatchNormalization())
model.add(ReLU())

model.add(Flatten())

model.add(Reshape((1, -1)))
model.add(LSTM(58, return_sequences=False))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(2, activation='sigmoid'))

model.build(input_shape=(129, 18, 3))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[ metrics.MeanSquaredError(name='my_mse'),
                                                                           metrics.AUC(name='my_auc'),])
model.summary()


In [ ]:
del model
gc.collect()

#notes for improvement:
# compare using bandpass filter (theta and alpha and beta)
# use optuna on validation
# EEGNET try

In [ ]:
print(finX_train.shape)
print(finX_test.shape)
print(finX_val.shape)

In [ ]:
from PIL import Image
import numpy as np
X_train = []
X_test = []
X_val = []

for i in range(21112):
    img = Image.fromarray(finX_train[i].astype(np.uint8))
    img_resized = img.resize((224, 224))
    X_train.append(np.array(img_resized))

for i in range(2484):
    img = Image.fromarray(finX_test[i].astype(np.uint8))
    img_resized = img.resize((224, 224))
    X_test.append(np.array(img_resized))

for i in range(1242):
    img = Image.fromarray(finX_val[i].astype(np.uint8))
    img_resized = img.resize((224, 224))
    X_val.append(np.array(img_resized))

In [ ]:
import keras
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(VGG16(weights='imagenet', input_shape=(224, 224, 3)))
for layer in model.layers:
  layer.trainable = False
model.add(Dense(2, activation='softmax'))
model.summary()

In [ ]:
from keras import metrics

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=[ metrics.MeanSquaredError(name='my_mse'),
                                                                           metrics.AUC(name='my_auc'),
                                                                           'accuracy'])

In [ ]:
X_train = np.array(X_train)
print(X_train.shape)
X_val = np.array(X_val)

In [ ]:
print(y_train.shape)
print(y_val.shape)

In [ ]:
import tensorflow as tf
y_train = tf.keras.utils.to_categorical(y_train, 2)
y_val = tf.keras.utils.to_categorical(y_val, 2)

In [ ]:
pip install keras-unet-collection

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from keras_unet_collection import models

# Define the input shape of your EEG spectrogram data
input_shape = (224, 224, 3)

# Create the U-Net model
unet_model = models.unet_2d(input_size=input_shape, n_labels=2, filter_num=[64, 128, 256, 512], batch_norm=True)

# Add a GlobalAveragePooling2D layer to reduce the spatial dimensions to 1x1
x = GlobalAveragePooling2D()(unet_model.output)

# Add a Dense layer with 2 units for binary classification
output = Dense(2, activation='sigmoid')(x)

# Create a new model that includes the U-Net model and the Dense layer
final_model = Model(inputs=unet_model.input, outputs=output)

# Compile the final model
final_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
final_model.summary()

In [ ]:
final_model.fit(X_train, y_train, epochs=10, verbose=1, batch_size=16, validation_data=(X_val, y_val))

In [ ]:
import selfeeg.models
import torch

In [ ]:
model = selfeeg.models.DeepConvNet(2, 3, 21112)

In [ ]:
import torch
print(finX_train.shape)
finX_train.reshape((2, 3, 21112))
finX_train_tensor = torch.from_numpy(finX_train)
print(finX_train_tensor.shape)

output = model(finX_train_tensor)

In [ ]:
import os
import tensorflow
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping


checkpoint = ModelCheckpoint("upbet_model.h5", monitor='val_', verbose=1, mode='auto', period=1)

es=tensorflow.keras.callbacks.EarlyStopping( monitor="val_loss", patience=5,
                                     verbose=1,  restore_best_weights=True)
rlronp=tensorflow.keras.callbacks.ReduceLROnPlateau( monitor="val_loss", factor=0.5, patience=3,
                                             verbose=1)
callbacks=[es, rlronp]
history = model.fit(finX_train, y_train,
            batch_size=16,
            epochs=20,
            verbose=1,
            validation_data=(finX_val, y_val),
            callbacks=[callbacks, checkpoint])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=15)

history = model.fit(X_train, y_train,
            batch_size=16,
            epochs=50,
            verbose=1,
            validation_data=(X_test, y_test),
            callbacks=[early_stopping])

In [ ]:
model.save('best_auc.h5')

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['my_auc'])
plt.plot(history.history['val_my_auc'])
plt.title('Model accuracy')
plt.ylabel('AUC')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
for i in range(2484):
    img = Image.fromarray(finX_test[i].astype(np.uint8))
    img_resized = img.resize((224, 224))
    X_test.append(np.array(img_resized))

In [ ]:
X_test = np.array(X_test)
print(X_test.shape)

(2484, 224, 224, 3)


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model


# best_mode = load_model('/content/drive/MyDrive/subhi/saved_data/sthuper_best_model.h5')
y_pred_prob = final_model.predict(X_test)

y_test_absolute = y_test[:, 1]
print(y_test_absolute)
y_pred_abs = np.where(y_pred_prob[:, 1] > 0.5, 1, 0)

fpr, tpr, thresholds = roc_curve(y_test_absolute, y_pred_abs)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_test_absolute, y_pred_abs)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Predicted Negative', 'Predicted Positive'],
            yticklabels=['True Negative', 'True Positive'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:

checkpoint = ModelCheckpoint("best_model.hdf5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto', period=1)

model.fit(X_train, dataset1_y, epochs=10,
          batch_size=32, verbose=1, callbacks= checkpoint)


#Reload model
model = load_model('best_model.hdf5')

#Continue training
dataset2_x = X_train[3000:]
dataset2_y = y_train[3000:]
model.fit(dataset2_x, dataset2_y, epoch=10, batch_size=200, verbose=2)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
model = keras.models.load_model("/content/drive/MyDrive/subhi/saved_data/best_model.hdf5")
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', verbose=1,
    save_best_only=True, mode='auto', period=1)


model.fit(X_train, y_train,
          batch_size=16,
          epochs=100,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=[checkpoint])


In [ ]:
import random
test = []
real = []
RND_NUMS = [random.randint(0, 17130) for _ in range(20)]
for r in RND_NUMS:
    test.append(X_train[r])
    real.append(y_train[r])
new_model = keras.models.load_model("/content/drive/MyDrive/subhi/saved_data/best_model.hdf5")


In [ ]:
predictions = new_model.predict(np.array(test))
pred = np.argmax(predictions, axis=1)
print(pred)
print(real)

In [ ]:
def data_generator(data, labels, batch_size):
    num_samples = len(data)
    indices = np.arange(num_samples)
    np.random.shuffle(indices)

    for i in range(0, num_samples, batch_size):
        batch_indices = indices[i:i+batch_size]
        batch_data = data[batch_indices]
        batch_labels = labels[batch_indices]
        yield batch_data, batch_labels

In [ ]:
import gc
batch_size = 100
epochs = 4
for epoch in range(epochs):
    for batch_data, batch_labels in data_generator(X_train, y, batch_size):
        model.train_on_batch(batch_data, batch_labels)
    del batch_data, batch_labels
    gc.collect()